# Sondera Harness + Strands

**Add policy enforcement to your Strands agents.**

This notebook shows how to use Sondera Harness with AWS Strands agents. You'll define Strands tools, write Cedar policies, and see how the harness evaluates each action before it executes.

We'll build a help desk agent that:
- **Allows** looking up tickets and customer information
- **Allows** updating ticket status and priority
- **Blocks** closing tickets without resolution
- **Blocks** assigning tickets to inactive agents

*This is a simulation: we evaluate what would be allowed, but no actual LLM calls are made.*

## Install

In [ ]:
%%capture
%pip install "sondera-harness[strands]"

## Define Strands Tools

In Strands, tools are Python functions decorated with `@tool`. Strands automatically generates JSON schemas from type hints.

In [ ]:
from strands.tools import tool


@tool
def get_ticket(ticket_id: str) -> str:
    """Look up a support ticket by ID."""
    return f"Ticket {ticket_id}: Status=Open, Priority=High, Customer=CUST001"


@tool
def get_customer_tickets(customer_id: str) -> str:
    """Get all tickets for a customer."""
    return f"Customer {customer_id} has 3 open tickets: T001, T002, T003"


@tool
def update_ticket_status(ticket_id: str, status: str, resolution: str) -> str:
    """Update a ticket's status. Resolution required when closing."""
    return f"Ticket {ticket_id} updated to {status}"


@tool
def assign_ticket(ticket_id: str, agent_id: str) -> str:
    """Assign a ticket to a support agent."""
    return f"Ticket {ticket_id} assigned to agent {agent_id}"


@tool
def escalate_ticket(ticket_id: str, reason: str, priority: int) -> str:
    """Escalate a ticket to higher priority. Priority 1-5 (5=highest)."""
    return f"Ticket {ticket_id} escalated to priority {priority}"


strands_tools = [
    get_ticket,
    get_customer_tickets,
    update_ticket_status,
    assign_ticket,
    escalate_ticket,
]
print(f"Defined {len(strands_tools)} tools: {[t.tool_name for t in strands_tools]}")

## Create Agent Metadata

Convert Strands tools to metadata for policy evaluation. Strands tools have a `tool_spec` attribute containing the name, description, and JSON schema.

In [ ]:
import json

from sondera import Agent, Parameter, Tool


def strands_tool_to_metadata(strands_tool):
    """Convert a Strands tool to metadata for policy evaluation."""
    spec = strands_tool.tool_spec
    input_schema = spec.get("inputSchema", {}).get("json", {})

    params = []
    for name, prop in input_schema.get("properties", {}).items():
        params.append(
            Parameter(
                name=name,
                description=prop.get("description", f"Parameter {name}"),
                type=prop.get("type", "string"),
            )
        )

    return Tool(
        name=spec["name"],
        description=spec["description"],
        parameters=params,
        parameters_json_schema=json.dumps(input_schema) if input_schema else None,
    )


tools_metadata = [strands_tool_to_metadata(t) for t in strands_tools]

agent_metadata = Agent(
    id="help_desk",
    provider_id="strands",
    name="help_desk",
    description="A help desk agent for managing support tickets.",
    instruction="Help customers with their support tickets. Be professional and efficient.",
    tools=tools_metadata,
)

print(
    f"Agent metadata for '{agent_metadata.name}' with {len(agent_metadata.tools)} tools"
)
for t in agent_metadata.tools:
    print(f"  - {t.name}: {len(t.parameters)} parameters")

## Define the Policy

Write Cedar policies to control what the agent can do. The namespace `help_desk` matches the agent name.

**Pattern:** Permit by default, then forbid specific things. `forbid` always wins over `permit`.

In [ ]:
from cedar import PolicySet

policy_set = PolicySet("""
// Allow looking up tickets
@id("allow-get-ticket")
permit(principal, action == help_desk::Action::"get_ticket", resource);

// Allow looking up customer tickets
@id("allow-get-customer-tickets")
permit(principal, action == help_desk::Action::"get_customer_tickets", resource);

// Allow assigning tickets
@id("allow-assign-ticket")
permit(principal, action == help_desk::Action::"assign_ticket", resource);

// Allow escalating tickets
@id("allow-escalate-ticket")
permit(principal, action == help_desk::Action::"escalate_ticket", resource);

// Allow updating ticket status by default (restricted below)
@id("allow-update-ticket-status")
permit(principal, action == help_desk::Action::"update_ticket_status", resource);

// Block closing tickets without a resolution
@id("forbid-close-without-resolution")
forbid(principal, action == help_desk::Action::"update_ticket_status", resource)
when {
  context has parameters &&
  context.parameters.status == "closed" &&
  context.parameters.resolution == ""
};

// Block assigning to inactive agents (agent IDs starting with "inactive_")
@id("forbid-assign-inactive-agent")
forbid(principal, action == help_desk::Action::"assign_ticket", resource)
when {
  context has parameters &&
  context.parameters.agent_id like "inactive_*"
};

// Block excessive escalation (priority > 5)
@id("forbid-excessive-escalation")
forbid(principal, action == help_desk::Action::"escalate_ticket", resource)
when {
  context has parameters &&
  context.parameters.priority > 5
};
""")

print(f"Policy loaded with {len(policy_set)} rules")

## Initialize the Harness

Create a `CedarPolicyHarness` for local policy evaluation. In production, you'd use `SonderaRemoteHarness` with `SonderaHarnessHook` to enforce policies automatically.

In [ ]:
from sondera import CedarPolicyHarness
from sondera.harness.cedar.schema import agent_to_cedar_schema

schema = agent_to_cedar_schema(agent_metadata)
harness = CedarPolicyHarness(policy_set=policy_set, schema=schema)
await harness.initialize(agent=agent_metadata)

print("Harness initialized and ready")

## How the Hook Works

In a real Strands agent, `SonderaHarnessHook` intercepts execution at each stage:

| Strands Event | Stage | What It Checks |
|:--------------|:------|:---------------|
| `BeforeInvocationEvent` | `PRE_RUN` | Session start, initialize trajectory |
| `BeforeModelCallEvent` | `PRE_MODEL` | Model requests |
| `AfterModelCallEvent` | `POST_MODEL` | Model responses |
| `BeforeToolCallEvent` | `PRE_TOOL` | Tool arguments |
| `AfterToolCallEvent` | `POST_TOOL` | Tool results |
| `AfterInvocationEvent` | `POST_RUN` | Session end, finalize trajectory |

Below, we'll simulate what the hook does by calling `adjudicate()` directly.

## Run It

Let's test what the agent is allowed to do. We call `adjudicate()` for each action, simulating what the hook would do at the `PRE_TOOL` stage.

In [ ]:
from sondera import Decision, Role, Stage, ToolRequestContent

# ANSI color codes
GREEN = "\033[92m"
RED = "\033[91m"
RESET = "\033[0m"


def check(description: str, result, expected: Decision):
    """Check result matches expected and print colored status."""
    color = GREEN if result.decision == Decision.ALLOW else RED
    status = result.decision.name
    icon = "\u2705" if result.decision == expected else "\u274c"
    print(f"{icon} {description} ({color}{status}{RESET})")
    if result.decision != expected:
        print(f"   Expected {expected.name}, got {result.decision.name}")
        if result.reason:
            print(f"   Reason: {result.reason}")


# =============================================================================
# ALLOWED OPERATIONS
# =============================================================================

# Looking up a ticket is allowed
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(tool_id="get_ticket", args={"ticket_id": "T001"}),
)
check("Look up ticket", result, Decision.ALLOW)

# Looking up customer tickets is allowed
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(tool_id="get_customer_tickets", args={"customer_id": "CUST001"}),
)
check("Look up customer tickets", result, Decision.ALLOW)

# Updating status to 'in_progress' is allowed
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="update_ticket_status",
        args={"ticket_id": "T001", "status": "in_progress", "resolution": ""},
    ),
)
check("Update status to in_progress", result, Decision.ALLOW)

# Closing with a resolution is allowed
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="update_ticket_status",
        args={
            "ticket_id": "T001",
            "status": "closed",
            "resolution": "Issue resolved by updating customer settings",
        },
    ),
)
check("Close with resolution", result, Decision.ALLOW)

# Assigning to an active agent is allowed
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="assign_ticket", args={"ticket_id": "T001", "agent_id": "agent_alice"}
    ),
)
check("Assign to active agent", result, Decision.ALLOW)

# Normal escalation is allowed
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="escalate_ticket",
        args={"ticket_id": "T001", "reason": "Customer is VIP", "priority": 4},
    ),
)
check("Escalate to priority 4", result, Decision.ALLOW)

# =============================================================================
# BLOCKED OPERATIONS
# =============================================================================

# Closing without resolution is blocked
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="update_ticket_status",
        args={"ticket_id": "T001", "status": "closed", "resolution": ""},
    ),
)
check("Close without resolution", result, Decision.DENY)

# Assigning to inactive agent is blocked
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="assign_ticket", args={"ticket_id": "T001", "agent_id": "inactive_bob"}
    ),
)
check("Assign to inactive agent", result, Decision.DENY)

# Excessive escalation is blocked
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="escalate_ticket",
        args={"ticket_id": "T001", "reason": "Urgent", "priority": 10},
    ),
)
check("Escalate to priority 10 (over limit)", result, Decision.DENY)

print(f"\n{GREEN}{'=' * 50}")
print(f"All policy evaluations complete!{RESET}")

## Using the Hook

In a real Strands agent, you'd use `SonderaHarnessHook` to automatically enforce policies. Here's how:

```python
from strands import Agent
from sondera.harness import SonderaRemoteHarness
from sondera.strands import SonderaHarnessHook

# Create harness (uses SONDERA_API_TOKEN from environment)
harness = SonderaRemoteHarness()

# Create hook
hook = SonderaHarnessHook(harness=harness)

# Create agent with hook
agent = Agent(
    system_prompt="You are a help desk agent.",
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    tools=[get_ticket, get_customer_tickets, update_ticket_status, ...],
    hooks=[hook],
)

# Run the agent - policies are automatically enforced
response = agent("Show me ticket T001")
```

**On denial:** The hook cancels the tool call and provides feedback to the agent, allowing it to inform the user or try a different approach.

## Try It Yourself

Experiment with the policy. Go back to **Define the Policy** and try:

- **Require reason for escalation**: Add a rule that forbids escalation when `context.parameters.reason == ""`
- **Limit assignments per ticket**: Track assignment count and forbid if too many reassignments
- **Time-based rules**: Block ticket updates outside business hours (requires adding timestamp to context)

Or modify the test cases below:

In [ ]:
def show(description: str, result):
    """Print colored result."""
    color = GREEN if result.decision == Decision.ALLOW else RED
    print(f"{description}: {color}{result.decision.name}{RESET}")


# Try escalating to exactly priority 5 (should be allowed)
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="escalate_ticket",
        args={"ticket_id": "T002", "reason": "Critical customer impact", "priority": 5},
    ),
)
show("Escalate to priority 5 (at limit)", result)

# Try priority 6 (just over limit)
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="escalate_ticket",
        args={"ticket_id": "T002", "reason": "Critical customer impact", "priority": 6},
    ),
)
show("Escalate to priority 6 (over limit)", result)

# Try assigning to another inactive agent pattern
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="assign_ticket",
        args={"ticket_id": "T003", "agent_id": "inactive_charlie"},
    ),
)
show("Assign to inactive_charlie", result)

## Next Steps

- [Quickstart](https://docs.sondera.ai/quickstart/) - Core concepts without framework integration
- [Writing Policies](https://docs.sondera.ai/writing-policies/) - Cedar syntax and patterns
- [LangGraph Integration](https://docs.sondera.ai/integrations/langgraph/) - Use with LangChain and LangGraph
- [ADK Integration](https://docs.sondera.ai/integrations/adk/) - Use with Google's Agent Development Kit